In [5]:
from fileinput import filename

import polars as pl
import polars.selectors as cs
import time
import chardet

from numpy.f2py.cfuncs import includes
from sqlalchemy.dialects.mssql.information_schema import columns

penguins = pl.read_csv("../data/penguins.csv")
print(penguins)

shape: (344, 9)
┌───────┬───────────┬───────────┬────────────────┬───┬───────────────┬─────────────┬────────┬──────┐
│ rowid ┆ species   ┆ island    ┆ bill_length_mm ┆ … ┆ flipper_lengt ┆ body_mass_g ┆ sex    ┆ year │
│ ---   ┆ ---       ┆ ---       ┆ ---            ┆   ┆ h_mm          ┆ ---         ┆ ---    ┆ ---  │
│ i64   ┆ str       ┆ str       ┆ str            ┆   ┆ ---           ┆ str         ┆ str    ┆ i64  │
│       ┆           ┆           ┆                ┆   ┆ str           ┆             ┆        ┆      │
╞═══════╪═══════════╪═══════════╪════════════════╪═══╪═══════════════╪═════════════╪════════╪══════╡
│ 1     ┆ Adelie    ┆ Torgersen ┆ 39.1           ┆ … ┆ 181           ┆ 3750        ┆ male   ┆ 2007 │
│ 2     ┆ Adelie    ┆ Torgersen ┆ 39.5           ┆ … ┆ 186           ┆ 3800        ┆ female ┆ 2007 │
│ 3     ┆ Adelie    ┆ Torgersen ┆ 40.3           ┆ … ┆ 195           ┆ 3250        ┆ female ┆ 2007 │
│ 4     ┆ Adelie    ┆ Torgersen ┆ NA             ┆ … ┆ NA            ┆ NA  

In [6]:
penguins = pl.read_csv("../data/penguins.csv", null_values="NA")
print(penguins)

shape: (344, 9)
┌───────┬───────────┬───────────┬────────────────┬───┬───────────────┬─────────────┬────────┬──────┐
│ rowid ┆ species   ┆ island    ┆ bill_length_mm ┆ … ┆ flipper_lengt ┆ body_mass_g ┆ sex    ┆ year │
│ ---   ┆ ---       ┆ ---       ┆ ---            ┆   ┆ h_mm          ┆ ---         ┆ ---    ┆ ---  │
│ i64   ┆ str       ┆ str       ┆ f64            ┆   ┆ ---           ┆ i64         ┆ str    ┆ i64  │
│       ┆           ┆           ┆                ┆   ┆ i64           ┆             ┆        ┆      │
╞═══════╪═══════════╪═══════════╪════════════════╪═══╪═══════════════╪═════════════╪════════╪══════╡
│ 1     ┆ Adelie    ┆ Torgersen ┆ 39.1           ┆ … ┆ 181           ┆ 3750        ┆ male   ┆ 2007 │
│ 2     ┆ Adelie    ┆ Torgersen ┆ 39.5           ┆ … ┆ 186           ┆ 3800        ┆ female ┆ 2007 │
│ 3     ┆ Adelie    ┆ Torgersen ┆ 40.3           ┆ … ┆ 195           ┆ 3250        ┆ female ┆ 2007 │
│ 4     ┆ Adelie    ┆ Torgersen ┆ null           ┆ … ┆ null          ┆ null

In [9]:
print(penguins.null_count().transpose(
    include_header=True, column_names=["null_count"]
))


shape: (9, 2)
┌───────────────────┬────────────┐
│ column            ┆ null_count │
│ ---               ┆ ---        │
│ str               ┆ u32        │
╞═══════════════════╪════════════╡
│ rowid             ┆ 0          │
│ species           ┆ 0          │
│ island            ┆ 0          │
│ bill_length_mm    ┆ 2          │
│ bill_depth_mm     ┆ 2          │
│ flipper_length_mm ┆ 2          │
│ body_mass_g       ┆ 2          │
│ sex               ┆ 11         │
│ year              ┆ 0          │
└───────────────────┴────────────┘


In [15]:
import chardet
def detect_encoding(filename: str) -> str:
    with open(filename, "rb") as f:
        raw_data = f.read()
        result = chardet.detect(raw_data)
        return result["encoding"]

print(detect_encoding("../data/directors.csv"))

EUC-JP


In [16]:
pl.read_csv("../data/directors.csv", encoding="EUC-JP")

name,born,country
str,i64,str
"""深作""",1930,"""日本"""
"""Verhoeven""",1938,"""オランダ"""
"""宮崎""",1942,"""日本"""
"""Tarantino""",1963,"""米国"""
